In [255]:
import altair as alt
import pandas as pd

df_apt=pd.read_csv('./data/부산_아파트실거래가_위도경도.csv',encoding='utf-8')

In [256]:
# 금액형 변수에서 쉼표 제거하고 숫자로 변환
for col in ["보증금액", "월세금액"]:
    df_apt[col] = pd.to_numeric(df_apt[col].astype(str).str.replace(",", ""), errors="coerce")

# dtype 지정
df_apt = df_apt.astype({
    "법정동": "string",
    "지번": "string",
    "단지명": "string",
    "거래연도": "Int64",
    "전용면적": "float64",
    "구별": "string",
    "보증금액": "float64",
    "월세금액": "float64",
    "주소지": "string",
    "전/월세": "string",
    "위도": "float64",
    "경도": "float64"
})

데이터를 전세데이터, 월세 데이터로 분할함

In [257]:
# 전세/월세 분리
df_jeonse = df_apt[df_apt["전/월세"] == "전세"].copy()
df_wolse  = df_apt[df_apt["전/월세"] == "월세"].copy()


# 2. Altair 활용한 자치구별 통계적 분석 및 변수간 상관관계 차트

### 1. 전세 분석
(1) 구별 평균 보증금

In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()

jeonse_bar = alt.Chart(df_jeonse).mark_bar().encode(
    x=alt.X("구별:N", sort="-y", title="자치구"),
    y=alt.Y("mean(보증금액):Q", title="평균 보증금(만원)"),
    color="구별:N",
    tooltip=["구별", alt.Tooltip("mean(보증금액):Q", format=",.0f")]
).properties(title="자치구별 평균 전세 보증금")
jeonse_bar

(2) 변수간 상관관계 heatmap

In [259]:
num_cols = ["전용면적", "보증금액",'층']
num_cols = [c for c in num_cols if c in df_apt.columns]

def corr_long(df, num_cols):
    corr = df[num_cols].corr(method='pearson')
    corr_df = corr.reset_index().melt('index')
    corr_df.columns = ['변수1', '변수2', '상관계수']
    return corr_df

jeonse_corr = corr_long(df_jeonse, num_cols)

In [260]:
def make_corr_heatmap(corr_df, title):
    base = alt.Chart(corr_df).encode(
        x=alt.X('변수1:N', sort=num_cols, title=None),
        y=alt.Y('변수2:N', sort=num_cols, title=None)
    )

    heat = base.mark_rect().encode(
        color=alt.Color('상관계수:Q',
                        scale=alt.Scale(scheme='redblue', domain=[-1, 1])),
        tooltip=['변수1', '변수2', alt.Tooltip('상관계수:Q', format=".2f")]
    )

    text = base.mark_text(baseline='middle', fontSize=12).encode(
        text=alt.Text('상관계수:Q', format=".2f"),
        color=alt.condition(
            "datum.상관계수 > 0.5 || datum.상관계수 < -0.5",
            alt.value('white'),
            alt.value('black')
        )
    )

    return (heat + text).properties(
        width=300,
        height=300,
        title=title
    )

In [261]:
jeonse_heatmap = make_corr_heatmap(jeonse_corr, "[전세] 변수 간 상관관계 Heatmap")

### 2. 월세 분석
(1) 구별 평균 월세금액

In [262]:
wolse_bar = alt.Chart(df_wolse).mark_bar().encode(
    x=alt.X("구별:N", sort="-y", title="자치구"),
    y=alt.Y("mean(월세금액):Q", title="평균 월세금액(만원)"),
    color="구별:N",
    tooltip=["구별", alt.Tooltip("mean(월세금액):Q", format=",.0f")]
).properties(title="자치구별 평균 월세금액")
wolse_bar

MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000).

Try enabling the VegaFusion data transformer which raises this limit by pre-evaluating data
transformations in Python.
    >> import altair as alt
    >> alt.data_transformers.enable("vegafusion")

Or, see https://altair-viz.github.io/user_guide/large_datasets.html for additional information
on how to plot large datasets.

alt.Chart(...)

(2) 변수간 상관관계 heatmap

In [263]:
num_cols = ["전용면적", "보증금액", "월세금액",'층']
num_cols = [c for c in num_cols if c in df_apt.columns]
wolse_corr  = corr_long(df_wolse, num_cols)
wolse_heatmap  = make_corr_heatmap(wolse_corr, "[월세] 변수 간 상관관계 Heatmap")

통합 차트

In [264]:
jeonse_heatmap | wolse_heatmap

alt.HConcatChart(...)

# 전세/제곱미터 가격의 월별 변화 Plotly 애니매이션 차트

In [265]:
df_jeonse.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26556 entries, 0 to 49179
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   법정동     26556 non-null  string 
 1   지번      26556 non-null  string 
 2   단지명     26556 non-null  string 
 3   거래연도    26556 non-null  Int64  
 4   전용면적    26556 non-null  float64
 5   구별      26556 non-null  string 
 6   보증금액    26556 non-null  float64
 7   월세금액    26556 non-null  float64
 8   주소지     26556 non-null  string 
 9   전/월세    26556 non-null  string 
 10  층       26556 non-null  int64  
 11  년월      26556 non-null  int64  
 12  위도      26556 non-null  float64
 13  경도      26556 non-null  float64
dtypes: Int64(1), float64(5), int64(2), string(6)
memory usage: 3.1 MB


In [266]:
import plotly.express as px

df=df_jeonse
df_jeonse["전세단가"] = df_jeonse["보증금액"] / df_jeonse["전용면적"]
df_jeonse["년월"] = df_jeonse["년월"].astype(str)
df_jeonse["년월"] = df_jeonse["년월"].str.slice(0,4) + "-" + df_jeonse["년월"].str.slice(4,6)

agg_j = df_jeonse.groupby(["구별", "년월"], as_index=False).agg(
        전세단가=("전세단가", "median"),
        전용면적=("전용면적", "median"),
        보증금_중앙값=("보증금액", "median"),
        거래건수=("전세단가", "size")
    )

In [267]:
fig = px.scatter(
    agg_j,
    x="전세단가",                 # 만원/㎡
    y="전용면적",                 # ㎡ (중앙값)
    size="거래건수",              # 버블 크기 = 해당 구×월 거래건수
    size_max=60,
    color="구별",
    hover_data=["보증금_중앙값", "거래건수"],
    log_x=True,                   # 단가 분포가 치우치므로 로그축 권장
    animation_frame="년월",       # 월별 애니메이션
    animation_group="구별",       # 동일 구는 같은 개체로 취급
    title="부산 전세/㎡(제곱미터) 가격의 월별 변화 (구별 집계)",
    width=800,
    height=550,
)

fig.update_layout(
    xaxis_title="전세 단가 (만원/㎡, log)",
    yaxis_title="전용면적 중앙값 (㎡)",
    legend_title_text="자치구",
)
fig.show()

In [268]:
import pandas as pd
import numpy as np

# 1) 월세만 사용 + 금액 숫자화
df_wolse["월세금액"] = pd.to_numeric(df_wolse["월세금액"].astype(str).str.replace(",", ""), errors="coerce")

# 2) 월세/㎡ 단가(만원/㎡)
df_wolse["월세단가"] = df_wolse["월세금액"] / df_wolse["전용면적"]

# 3) 년월 라벨(YYYY-MM) 만들기
df_wolse["년월"] = df_wolse["년월"].astype(str)
df_wolse["년월"] = df_wolse["년월"].str.slice(0,4) + "-" + df_wolse["년월"].str.slice(4,6)

# 4) 구별×월 단위 집계 (중앙값 기준이 일반적으로 안정적)
agg_w = (
    df_wolse.groupby(["구별", "년월"], as_index=False)
    .agg(월세단가=("월세단가", "median"),
         거래건수=("월세단가", "size"))
)

# 월, 구 순서 고정
months  = sorted(agg_w["년월"].unique())
gus     = sorted(agg_w["구별"].unique())


In [269]:
import plotly.graph_objects as go
import numpy as np

# 1) 피벗: rows=구별, cols=년월, values=월세단가(중앙값)
pivot = (
    agg_w.pivot(index="구별", columns="년월", values="월세단가")
    .reindex(index=gus, columns=months)
)

# Plotly Surface는 숫자 좌표가 안정적 → 축용 숫자 좌표 생성
x_pos = np.arange(len(months))   # 월 인덱스 (x축)
y_pos = np.arange(len(gus))      # 구 인덱스 (y축)
Z = pivot.values.astype(float)   # z축 = 월세단가

fig_surf = go.Figure(data=[
    go.Surface(
        z=Z,
        x=x_pos,
        y=y_pos,
        colorscale="Viridis",
        colorbar=dict(title="월세단가(만원/㎡)")
    )
])

fig_surf.update_layout(
    title="자치구별 월세/㎡ 가격 변화 (3D Surface: 중앙값)",
    width=900, height=650,
    scene=dict(
        xaxis=dict(title="년월", tickmode="array",
                   tickvals=x_pos, ticktext=months),
        yaxis=dict(title="구별", tickmode="array",
                   tickvals=y_pos, ticktext=gus),
        zaxis=dict(title="월세단가(만원/㎡)")
    )
)

fig_surf.show()


# 자치구별 아파트 전세/제곱미터 가격에 대한 단계 구분도- 지도시각화

In [270]:
import pandas as pd
import numpy as np
import pydeck as pdk

In [271]:
import pandas as pd
gu_json=pd.read_json(r'.\data\N3A_G0100000.json')
df_gu_geo=pd.DataFrame()
#26은 부산 법정동코드
busan_features = [f for f in gu_json['features'] if f['properties']['BJCD'].startswith('26')]
# DataFrame 생성
df_gu_geo = pd.DataFrame({
    '구별': [f['properties']['NAME'] for f in busan_features],
    '좌표': [f['geometry']['coordinates'] for f in busan_features]
})
df_gu_geo.head()

,구별,좌표
0,중구,"[[[129.04432032308114, 35.10827279057366], [12..."
1,서구,"[[[129.01216676817145, 35.13746585357049], [12..."
2,동구,"[[[129.0645382268616, 35.141415448612726], [12..."
3,영도구,"[[[129.06250865152413, 35.10283168642716], [12..."
4,부산진구,"[[[129.04996448421414, 35.193703412527526], [1..."


In [272]:
df_avg_price=agg_j.merge(
    df_gu_geo,
    on='구별',
    how='left'
)

df_avg_price["보증금_중앙값"] = df_avg_price["보증금_중앙값"] / 10000
df_avg_price['가격_문자']=df_avg_price['보증금_중앙값'].apply(lambda x:f'{x:.2f}')
df_avg_price['정규화가격']=df_avg_price['전세단가']/df_avg_price['전세단가'].max()
df_avg_price.head()
df_avg_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   구별       160 non-null    object 
 1   년월       160 non-null    object 
 2   전세단가     160 non-null    float64
 3   전용면적     160 non-null    float64
 4   보증금_중앙값  160 non-null    float64
 5   거래건수     160 non-null    int64  
 6   좌표       160 non-null    object 
 7   가격_문자    160 non-null    object 
 8   정규화가격    160 non-null    float64
dtypes: float64(4), int64(1), object(4)
memory usage: 11.4+ KB


In [273]:
import pandas as pd

# 주요 관공서 매핑 (요약판) gpt사용
gu_offices = {
    "중구": "중구청, 부산중부경찰서, 중부소방서, 부산세관, 중구보건소",
    "서구": "서구청, 서부경찰서, 서부소방서, 부산의료원, 서구보건소",
    "동구": "동구청, 동부경찰서, 동구보건소, 부산역",
    "영도구": "영도구청, 영도경찰서, 영도소방서, 영도보건소",
    "부산진구": "부산진구청, 부산진경찰서, 서면세무서, 부산진보건소",
    "동래구": "동래구청, 동래경찰서, 동래소방서, 동래세무서, 동래보건소",
    "남구": "남구청, 남부경찰서, 남부소방서, 남구보건소",
    "북구": "북구청, 북부경찰서, 북부소방서, 북구보건소",
    "해운대구": "해운대구청, 해운대경찰서, 해운대소방서, 해운대세무서, 해운대보건소",
    "사하구": "사하구청, 사하경찰서, 사하소방서, 사하보건소",
    "금정구": "금정구청, 금정경찰서, 금정소방서, 금정보건소",
    "강서구": "강서구청, 강서경찰서, 강서소방서, 신항만세관, 강서보건소",
    "연제구": "연제구청, 부산지방경찰청, 연제세무서, 연제보건소",
    "수영구": "수영구청, 수영경찰서, 수영소방서, 수영보건소",
    "사상구": "사상구청, 사상경찰서, 사상소방서, 사상보건소",
    "기장군": "기장군청, 기장경찰서, 기장소방서, 국립부산과학관, 기장보건소"
}

# 주요 관공서 DataFrame 생성
df_office = pd.DataFrame(list(gu_offices.items()), columns=["구별", "주요관공서"])

# 기존 df와 병합 (구별 기준)
df_avg_price = df_avg_price.merge(df_office, on="구별", how="left")

# 확인
df_avg_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   구별       160 non-null    object 
 1   년월       160 non-null    object 
 2   전세단가     160 non-null    float64
 3   전용면적     160 non-null    float64
 4   보증금_중앙값  160 non-null    float64
 5   거래건수     160 non-null    int64  
 6   좌표       160 non-null    object 
 7   가격_문자    160 non-null    object 
 8   정규화가격    160 non-null    float64
 9   주요관공서    160 non-null    object 
dtypes: float64(4), int64(1), object(5)
memory usage: 12.6+ KB


In [274]:
import pandas as pd
from copy import deepcopy

def explode_multipolygon(df, coord_col="좌표"):
    rows = []
    for _, r in df.iterrows():
        coords = r[coord_col]
        # MultiPolygon: coords[0][0][0] 가 list/tuple
        is_multi = isinstance(coords[0][0][0], (list, tuple))
        if is_multi:
            for poly in coords:  # 각 폴리곤 단위로 행 분리
                rr = deepcopy(r)
                rr[coord_col] = poly
                rows.append(rr)
        else:
            rows.append(r)
    return pd.DataFrame(rows, columns=df.columns)

df_plot = explode_multipolygon(df_avg_price)

In [275]:
layer_polygon=pdk.Layer(
    'PolygonLayer',
    data=df_plot,
    get_polygon='좌표',
    opacity=0.8,
    get_fill_color='[0,255*정규화가격,0]',
    pickable=True,
    auto_highlight=True
)

tooltip={
    "html": (
        "<b>{구별}</b><br/>"
        "평균가격: {가격_문자}억원<br/>"
        "주요 관공서: {주요관공서}"
    ),
    'style':{'color':'white','backgroundColor': 'steelblue'}}


view_state_2d=pdk.ViewState(
    longitude=129.08,
    latitude=35.17,
    zoom=10.5,
    ptich=0
)

deck_polygon=pdk.Deck(
    layers=[layer_polygon],
    initial_view_state=view_state_2d,
    map_style='dark',
    tooltip=tooltip
)

deck_polygon.to_html(
    'pydeck_apt_polygon_2d.html',
    notebook_display=False,
    open_browser=True
)

In [276]:
layer_polygon=pdk.Layer(
    'PolygonLayer',
    data=df_plot,
    get_polygon='좌표',
    stroked=True,
    extruded=True,
    get_elevation='정규화가격*3000',
    get_fill_color='[0,255*정규화가격,255]',
    pickable=True,
    auto_highlight=True
)

tooltip={
    'html':'<b>{구별}</b><br/>평균가격: {가격_문자}억원',
    'style':{'color':'white','backgroundColor': 'steelblue'}}

view_state_3D=pdk.ViewState(
    longitude=129.08,
    latitude=35.17,
    zoom=10.5,
    pitch=50
)

deck_polygon=pdk.Deck(
    layers=[layer_polygon],
    initial_view_state=view_state_3D,
    map_style='dark',
    tooltip=tooltip
)

deck_polygon.to_html(
    'pydeck_apt_polygon_2d.html',
    notebook_display=False,
    open_browser=False
)